In [3]:
import glob
import os

import numpy as np
import pandas as pd

# setting notebook
pd.set_option("display.max_columns", None)

In [4]:
# go to data-joined folder
%cd data-joined

C:\Users\user\Desktop\gte\notebooks\data-joined


In [5]:
%run xuat_combined.ipynb 
%run po_combined.ipynb 
%run dmsp.ipynb
%run dmkh.ipynb
#go back the tree
%cd ..

C:\Users\user\Desktop\gte\notebooks


In [6]:
# make copy
xuat = xuat_joined.copy()
po = po_joined.copy()
sp = dmsp_full.copy()
kh = dmkh_full.copy()

# PO

In [7]:
#drop cols
cols_used = ['Mã phiếu đề xuất\n(Code)',
 'Mã khách hàng\n(Customer Code)',
 'Mã chủ hàng',
 'Ngày về\n(Receiving Date)',
 'Hạn chót giao hàng\n(Delivery Deadline)',
 'Ngày thực tế giao hàng',
 'Chứng từ KH ký nhận',
 'Chứng từ khác',
 'Tình trạng đơn hàng',
 'Kho xuất',
 'Loại phiếu']

po_selected_cols = po[cols_used]

In [8]:
po_replace = po_selected_cols.copy()

#Replace 'Chua thuc hien' => 'Da nhat hang'
po_replace.loc[(po_replace['Tình trạng đơn hàng'].str.contains('Chưa thực hiện')) 
                & (po_replace['Chứng từ khác'].str.contains('Đã nhặt hàng')),'Tình trạng đơn hàng'] = 'Đã nhặt hàng'

#Replace 'De xuat' => 'Don'
po_replace['Loại phiếu'] = po_replace['Loại phiếu'].str.replace(r'Đề xuất', 'Đơn', regex=True)

In [9]:
# make a copy
po_cal = po_replace.copy()
# create month
po_cal['month'] = pd.DatetimeIndex(po_cal['Ngày về\n(Receiving Date)']).month
# impute date diff invoice

diff_ct_yes = (po_cal['Chứng từ KH ký nhận'] - po_cal['Ngày thực tế giao hàng']).dt.days #da hoan
diff_ct_no = (pd.Timestamp.now().normalize() - po_cal['Ngày thực tế giao hàng']).dt.days #chua hoan

conditions = [
    po_cal['Tình trạng đơn hàng']=='Đã bàn giao chứng từ',
    po_cal['Tình trạng đơn hàng']=='Đã giao hàng'
]
choices = [diff_ct_yes, diff_ct_no]

po_cal['datediff_chungtu'] = np.select(conditions, choices, default=np.nan)

# impute date diff delivery

diff_deli = (po_cal['Ngày thực tế giao hàng'] - po_cal['Ngày về\n(Receiving Date)']).dt.days
po_cal['datediff_giaohang'] = np.where(diff_deli<0, np.nan, diff_deli)

In [10]:
po_filter = po_cal.copy()
#Filter Warehouse not contains
values_to_remove = ['Online','Hyperion','hư hỏng','xử lý','Hư Hỏng','Upbase','khuyến mãi','Tồn hàng','hàng hóa'] 

pattern = '|'.join(values_to_remove)

po_filter_kho = po_filter.loc[~po_filter['Kho xuất'].str.contains(pattern, case=False)]
#Filter Ma chu hang HP, RW
values_to_remove_mch = ['HP','RW'] 

pattern_mch = '|'.join(values_to_remove_mch)

po_filter_mch = po_filter_kho.loc[po_filter_kho['Mã chủ hàng'].str.contains(pattern_mch, case=False)]
po_processed = po_filter_mch.copy()

# BCKT

In [11]:
# merge
xuat_dmsp = xuat.merge(sp, left_on='Mã vật tư\n(Goods Code)', right_on='PRODUCT CODE', how='left')
xuat_dmsp['CBM/Unit'] = xuat_dmsp['CBM/Unit']*xuat_dmsp['Số lượng\n(Quantily)']
xuat_dmsp['GrossWeightProductg'] = xuat_dmsp['GrossWeightProductg']*xuat_dmsp['Số lượng\n(Quantily)']
# aggregate 
xuat_dmsp = xuat_dmsp.groupby(['Mã phiếu đề xuất', 'Mã chủ hàng_x']).agg(
      {
        'CBM/Unit': 'sum',
        'GrossWeightProductg': 'sum',
        'Số lượng\n(Quantily)': 'sum',
        'Mã vật tư\n(Goods Code)': 'count',
        'Tiền hàng\n(Sub Total)': 'sum'
      }
)
# rename cols
dict_rename = {
    'CBM/Unit': 'cbm_po',
     'GrossWeightProductg': 'kg_po',
     'Số lượng\n(Quantily)': 'unit_po',
     'Mã vật tư\n(Goods Code)': 'sku_po',
     'Tiền hàng\n(Sub Total)': 'amount_po'
}

xuat_dmsp.rename(columns=dict_rename, inplace=True)
xuat_processed = xuat_dmsp.copy()

## Create data

In [12]:
# Create data region
data = [
    ["Kho Hà Nội", "Miền Bắc"],
    ["Kho Phạm Văn Đồng", "Miền Nam"],
    ["Kho Bình Tân", "Miền Nam"],
    ["Kho Bùi Thị Xuân", "Miền Nam"],
    ["Kho Đà Nẵng", "Miền Trung"],
    ["Kho Thuận An", "Miền Nam"],
    ["Kho Bình Hòa", "Miền Nam"],
    ["Kho thuê dịch vụ - Daso", "Miền Nam"],
    ["Kho Woojin Bắc Ninh", "Miền Bắc"],
    ["Kho LogisValley - Long Biên", "Miền Bắc"],
    ["Kho thuê ngoài Emskip Q9", "Miền Nam"],
    ["Kho LogisticValey - Long Biên", "Miền Bắc"],
    ["Kho Hateco Long Biên", "Miền Bắc"],
    ["Kho Gamuda Hà Nội", "Miền Bắc"],
    ["Kho thuê dịch vụ Daso", "Miền Nam"],
    ['Kho LogisticValey Long Biên',"Miền Bắc"]
]

# Create the pandas DataFrame
df_rgn = pd.DataFrame(data, columns=["Kho", "Region"])
data_in_sub = [
    ["Kho Hà Nội", "Hà Nội", "Nội thành"],
    ["Kho Phạm Văn Đồng", "Hồ Chí Minh", "Nội thành"],
    ["Kho Bình Tân", "Hồ Chí Minh", "Nội thành"],
    ["Kho Bùi Thị Xuân", "Bình Dương", "Nội thành"],
    ["Kho Đà Nẵng", "Đà Nẵng", "Nội thành"],
    ["Kho Thuận An", "Bình Dương", "Nội thành"],
    ["Kho Bình Hòa", "Hồ Chí Minh", "Nội thành"],
    ["Kho thuê dịch vụ - Daso", "Hồ Chí Minh", "Nội thành"],
    ["Kho Woojin Bắc Ninh", "Hà Nội", "Nội thành"],
    ["Kho LogisValley - Long Biên", "Hà Nội", "Nội thành"],
    ["Kho thuê ngoài Emskip Q9", "Hồ Chí Minh", "Nội thành"],
    ["Kho LogisticValey - Long Biên", "Hà Nội", "Nội thành"],
    ["Kho Hateco Long Biên", "Hà Nội", "Nội thành"],
    ["Kho Gamuda Hà Nội", "Hà Nội", "Nội thành"],
    ["Kho thuê dịch vụ Daso", "Hồ Chí Minh", "Nội thành"],
    ['Kho LogisticValey Long Biên',"Hà Nội", "Nội thành"]
]

# Create the pandas DataFrame
df_in_sub = pd.DataFrame(data_in_sub, columns=["Kho xuất", "Des", "in_sub"])

## Merge

In [13]:
# merge po - bckt
po_xuat = po_processed.merge(
    xuat_processed, left_on=['Mã phiếu đề xuất\n(Code)', 'Mã chủ hàng'], right_on=['Mã phiếu đề xuất', 'Mã chủ hàng_x'], how="left")
# merge region
po_region = po_xuat.merge(df_rgn, left_on="Kho xuất", right_on="Kho", how="left").drop('Kho', axis=1)
# merge DMKH
po_kh = po_region.merge(
    kh[['Customer Code', 'Account', 'Des']], left_on="Mã khách hàng\n(Customer Code)", right_on="Customer Code", how="left").drop('Customer Code', axis=1)
# merge inner/suburbs data
po_insub = po_kh.merge(df_in_sub, on=["Kho xuất", "Des"], how="left")
po_insub.loc[~po_insub.Des.isna(), "in_sub"] = po_insub.in_sub.fillna("Ngoại thành")
conditions = [
    po_insub["datediff_chungtu"] < 0,
    po_insub["datediff_chungtu"] <= 6,
    po_insub["datediff_chungtu"] <= 14,
    po_insub["datediff_chungtu"] <= 30,
    po_insub["datediff_chungtu"] > 30,
]
choices = ["Invalid_days", "0 - 6 ngày", "7 - 14 ngày", "15 - 30 ngày", "Trễ"]

po_insub["day_level"] = np.select(conditions, choices, default=np.nan)
po_run = po_insub.copy()
po_run['Mã phiếu đề xuất\n(Code)'] = po_run['Mã phiếu đề xuất\n(Code)'] + po_run['Mã chủ hàng']
dict_rename = {
    "Mã phiếu đề xuất\n(Code)": "ID_PO",
    "Mã khách hàng\n(Customer Code)": "ID_cus",
    "Ngày về\n(Receiving Date)": "date_receive",
    "Hạn chót giao hàng\n(Delivery Deadline)": "deli_dl",
    "Tiền hàng\n(Sub Total)": "amount_po",
}

po_run.rename(columns=dict_rename, inplace=True)

In [14]:
import datetime
file_path = '../data/processed/'
namefile = f'PO_{datetime.datetime.now().strftime("%H%M_%Y%m%d")}.csv'
po_run.to_csv(file_path+namefile, index=False, encoding="utf-8-sig")


In [15]:
po_run[po_run.Region.isnull()]['Kho xuất'].unique()

array([], dtype=object)